[![Julia](../img/julia-logo-color.png)](https://julialang.org/)

# Seminario. Introducción al lenguaje Julia

## 12. Ámbito de variables

## Objetivos

- Ámbito de una variable
- Ámbito global
- Ámbito local
- Constantes

## Ámbito de una variable

El ámbito de una variable es la región del código en la que ésta es visible. El ámbito permite resolver conflictos entre nombres de variables pudiendo utilizar el mismo identificador para objetos diferentes, por ejemplo con dos funciones que usan el mismo nombre para alguno de sus argumentos. En Julia las regiones de código que marcan el ámbito de una variable están asociadas a las siguientes construcciones

|  **Construcción**  |  **Tipo de ámbito**  |  **Permitido en**  |
| :----------------- | :------------------- | :----------------- | 
| ```module```       |  _global_            |      _global_      |
| ```struct```       |  _local (soft)_      |      _global_      |
| ```for```, ```while```, ```try``` |  _local (soft)_     |  _global_, _local_  |
| ```macro```        |  _local (hard)_     |      _global_      |
| funciones, bloques ```do, let``` |  _local (hard)_     |  _global_, _local_  |

## Ámbito global

El ámbito global está asociado a los modulos. Cada módulo define su propia ámbito global. Por ejemplo, 

In [ ]:
module A
    a = 1  # Variable global en el ámbito del modulo A
end;

La variable **a** definida en el módulo A, es distinta a cualquier otra variable con el mismo nombre definida en otro ámbito, 

In [ ]:
a = 2
module A
    a = 1  # Variable global en el ámbito del modulo A
end;
println(a)

De esta manera, las funciones definidas en un módulo acceden a las variables globales definidas en él

In [4]:
module A
    a = 1
    imprime() = a    # se refiere a la variable global a del modulo A
end;
a = 2         # Esta variable es distinta a la del modulo A
A.imprime()

1

Los modulos pueden anidarse, y se puede acceder a las variables globales del módulo "interno" con el acceso
```julia
    *nombre_modulo*.*variable*
```

In [5]:
module B
    module C
        c = 2 
    end
    b = C.c   # Acceso al contenido del modulo C
end;    

Como se ha mencionado, cada módulo define un nuevo ámbito global, aunque puede usar las variables definidas en otros módulos generalmente mediante las sentencias ```using``` o ```import```. Si en el módulo B queremos acceder a las variables de A, tenemos que importarlo de la siguiente manera

In [6]:
module B
    module C
        c = 2
    end
    b = C.c      
    import ..A   # Modulo A disponible
    d = A.a
end; 

En los entornos interactivos como el REPL o los IJulia notebooks, todas las variables y módulos se declaran o definen en el módulo ```Main```, por lo que es posible acceder a su contenido directamente usando el acceso cualificado apropiado sin necesidad de utilizar ```import``` o ```using```,

In [7]:
println("Variable d = $(B.d)") 

Variable d = 1


In [8]:
println("Variable c = $(B.C.c)") 

Variable c = 2


Obsérvese esta otra forma de acceder a la variable **c**,

In [14]:
import .B.C
C.c

2

## Ámbito local